In [11]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

## Task1

빈 칸을 채워주세요!

단계별 output이 github 파일에는 남아있으니 그 output과 동일한 형태인지 확인하면서 진행해주시면 됩니다~

In [ ]:
#1. 생성할 문장 데이터

sentence = ("Brick walls are there for a reason and you must not think "
            "that the brick walls aren't there to keep us out, but rather "
            "in this way that the brick walls are there to show us how badly we want things.")

In [ ]:
#2. 문자 집합 만들기
world_set = list(set(sentence))

## 문제(1): 각 문자에 정수 인코딩 (공백도 하나의 원소로 포함)
vocab = {char: i for i, char in enumerate(world_set, start=0)}
# enumerate(world_set): world_set이라는 문자 집합에 대해 순회하면서 각 문자에 대해 인덱스와 함께 순회
# char: i: 딕셔너리의 키-값 쌍

In [ ]:
print(vocab)

{'k': 0, 'r': 1, 'h': 2, 'w': 3, 'd': 4, 'n': 5, 't': 6, ',': 7, '.': 8, 'B': 9, 'c': 10, 'b': 11, 'm': 12, ' ': 13, 'y': 14, "'": 15, 'i': 16, 'p': 17, 'e': 18, 'o': 19, 'a': 20, 'u': 21, 'f': 22, 'g': 23, 'l': 24, 's': 25}


In [ ]:
#3. 문자 집합 크기 확인

vocab_size = len(vocab)
print('문자 집합 크기 : {}'.format(vocab_size))

문자 집합 크기 : 26


In [ ]:
#4. 하이퍼 파라미터 설정(자유롭게 수정해보세요!)

hidden_size = vocab_size # 같아야 하는 것 확인!
sequence_length = 8  # 너무 길거나 너무 짧게 잡으면 안됩니다!
learning_rate = 0.01

In [ ]:
#5. seqence 길이 단위 자르기

# 데이터 구성을 위한 리스트
x_data = []
y_data = []

## 문제(2): 반복문 내에서의 인덱싱을 사용하여 sequence_length 값 단위로 샘플을 잘라 데이터 만들기, y_str은 x_str은 한 칸씩 쉬프트된 sequnce

for i in range(0, len(sentence) - sequence_length):
  x_str = sentence[i:i+sequence_length]
  y_str = sentence[i+1:i+sequence_length+1]
  print(i, x_str, "->", y_str)

  # x_str과 y_str이 문자집합에 해당하는 인덱스를 각각 x_data, y_data에 append
  x_data.append([vocab[c] for c in x_str])
  y_data.append([vocab[d] for d in y_str])

0 Brick wa -> rick wal
1 rick wal -> ick wall
2 ick wall -> ck walls
3 ck walls -> k walls 
4 k walls  ->  walls a
5  walls a -> walls ar
6 walls ar -> alls are
7 alls are -> lls are 
8 lls are  -> ls are t
9 ls are t -> s are th
10 s are th ->  are the
11  are the -> are ther
12 are ther -> re there
13 re there -> e there 
14 e there  ->  there f
15  there f -> there fo
16 there fo -> here for
17 here for -> ere for 
18 ere for  -> re for a
19 re for a -> e for a 
20 e for a  ->  for a r
21  for a r -> for a re
22 for a re -> or a rea
23 or a rea -> r a reas
24 r a reas ->  a reaso
25  a reaso -> a reason
26 a reason ->  reason 
27  reason  -> reason a
28 reason a -> eason an
29 eason an -> ason and
30 ason and -> son and 
31 son and  -> on and y
32 on and y -> n and yo
33 n and yo ->  and you
34  and you -> and you 
35 and you  -> nd you m
36 nd you m -> d you mu
37 d you mu ->  you mus
38  you mus -> you must
39 you must -> ou must 
40 ou must  -> u must n
41 u must n ->  must no
42

In [ ]:
# 출력해서 한 칸씩 쉬프트된 것 확인하기!

print(x_data[0])
print(y_data[0])

[9, 1, 16, 10, 0, 13, 3, 20]
[1, 16, 10, 0, 13, 3, 20, 24]


numpy의 eye란?
NumPy 라이브러리에서 제공하는 함수 중 하나 -> 단위 행렬(Identity matrix)을 생성하는 데 사용
단위 행렬은 주 대각선이 모두 1이고, 나머지 요소가 모두 0인 정사각 행렬

np.eye(N, M=None, k=0, dtype=<class 'float'>, order='C') 형태


In [ ]:
##6. 입력 시퀀스에 대해 원핫인코딩 수행

## 문제(4) : x_data를 원핫인코딩 > numpy의 eye를 쓸 수 있지 않을까?
x_one_hot = [np.eye(vocab_size)[x_seq] for x_seq in x_data]
x_one_hot1 = [np.identity(vocab_size)[x_seq] for x_seq in x_data]
# vocab size의 단위행렬 생성
# x_seq에 해당하는 행 선택해서, x_seq에 해당하는 인덱스의 원소만 1 나머지는 0으로 만듦
# 겉의 [ ]는 리스트 컴프리헨션 (리스트화)
# identity는 대각선 위치에만 1이 있는 행렬 생성. 사실상 동일해 보임

##7. 입력 데이터, 레이블데이터 텐서로 변환

## 문제(5) : x_one_hot과 y_data 텐서로 변환 : 둘 다 같은 형식의 텐서로 변환하면 될까?? (FloatTensor, LongTesor 중 맞는 것은?)
X = torch.FloatTensor(x_one_hot)
Y = torch.LongTensor(y_data)
# 둘 다 같은 형식의 텐서라면?

torch.FloatTensor는 부동 소수점 텐서를, torch.LongTensor는 정수 텐서를 생성하는 함수

https://jjuke-brain.tistory.com/entry/PyTorch-%EC%9D%B5%ED%9E%88%EA%B8%B0-Recurrent-Neural-Network-RNN-2

In [ ]:
##8. 크기 확인
print('훈련 데이터의 크기 : {}'.format(X.shape))
print('레이블의 크기 : {}'.format(Y.shape))

훈련 데이터의 크기 : torch.Size([190, 8, 26])
레이블의 크기 : torch.Size([190, 8])


In [ ]:
##9.원핫인코딩 결과 샘플 확인하기
print(X[0])

tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 1., 0., 0., 0., 0., 0.]])


In [ ]:
##10. 레이블 데이터 샘플 확인하기
print(Y[0])

tensor([ 1, 16, 10,  0, 13,  3, 20, 24])


https://jjuke-brain.tistory.com/entry/PyTorch-%EC%9D%B5%ED%9E%88%EA%B8%B0-Recurrent-Neural-Network-RNN-2

In [ ]:
##11. RNN 모델 구현

##문제(6) : 기본 pytorch 인자 넣기 연습 + forward 채우기
### 조건 : rnn layer 2개 쌓기 + 마지막은 fc layer
### batch_fisrt 설정 필요할까? (유튜브 강의 참고)

class Net(torch.nn.Module):
  def __init__(self, input_dim, hidden_dim, layers):
    super(Net, self).__init__()
    self.rnn = torch.nn.RNN(input_dim, hidden_dim, num_layers=layers, batch_first=True)
    self.fc = torch.nn.Linear(hidden_dim, hidden_dim, bias=True)

  def forward(self, x):
    x, _status = self.rnn(x)
    x = self.fc(x)
    return x

In [ ]:
net = Net(vocab_size, hidden_size, 2)

In [ ]:
##12. loss function
criterion = torch.nn.CrossEntropyLoss()

##13. optimizer
optimizer = optim.Adam(net.parameters(), learning_rate)

In [ ]:
##14. 출력 크기 점검
outputs = net(X)
print(outputs.shape)

torch.Size([190, 8, 26])


In [ ]:
##15. Training 시작

for i in range(100): # 100번의 epoch
    optimizer.zero_grad() # 매 반복마다 그래디언트 초기화
    outputs = net(X) # 모델의 출력 계산
    ##문제(7) : outputs, Y 형태 그대로 넣으면 안되죠. view 함수를 이용해 loss값을 계산해봅시다.
    loss = criterion(outputs.view(-1, vocab_size), Y.view(-1)) # view 함수로 텐서 형태 변환, view(-1)로 펼쳐주기
    loss.backward() # 역전파
    optimizer.step() # 최적화 알고리즘 사용, 파라미터 업데이트

    #16. 예측결과 확인
    results = outputs.argmax(dim=2)
    predict_str = ""
    for j, result in enumerate(results):
        if j == 0: # 처음에는 예측 결과를 전부 가져오기
            predict_str += ''.join([world_set[t] for t in result])
        else: # 그 다음에는 마지막 글자만 반복 추가
            predict_str += world_set[result[-1]]

    print(predict_str)

y.ys.t.'.syt'.yy.yy.yy.'.y'.'y..'y.'.sy.sy.y''.yy.byyddy.d.dd..y.dy.'ys.t.'.syt'.yy'byydddyy.'ydkydy.'y'.'.y..'.d'.dy.yyyt..yy.y'.s..'..y.dy.'ys.t.'.syt'.yy.yy.yy.'yyts.'.'yy'.'.'.sys.s..'..yyddy.y
sttstttsssstststttstsststssttsstsstttsststttssttssttsttsttttsttststtttstttsssststsstttststsstssttsttsststststtsttsttstststtttsttststtsttststtttstttsssstststttstsstssttstststtststssssttsttsttttttstt
stat tt tssttt ttast tt   t      at t  t t tt tta tttatat t  t tattt at tt tssttt a  ttat ttt t t t t ttt t tt t  t s  tattat tt t ta t sattt at tt tssttt ttast tt ttatt t ta t t  s ttstt tt tatat 
st t                                     t t                       t  t                             t                  t       t            t  t                          t      t     t  t          
tt                                                                                                                                                                                                   
t         

In [ ]:
predict_str

"rick walls are there tor a reason and you must not think that the brick walls are 't there to seep us out, but rather in this way that the brick walls are there to show us how badly we want thinks'"

결과가 어떤가요?? 마지막 에폭의 문장이 그럴싸한가요?
-> t와 f의 구분, brick에서의 b (가장 첫 문자)

## Task2

위 sentence는 제가 임의로 생성한 문장들입니다.

마음에 드시는 문구 가져오셔서 문장이 어떻게 생성되는지 확인해보세요!

영어가 아닌 한국어로 시도해보는 것도 좋겠죠?

수정이 많이 필요(토큰화 등) 할 수 있으나 한번 시도해보시는 것 권장드립니다 :)

위 베이스라인은 어디든 수정하셔도 좋고 조금 더 자연스러운 문장이 나올 수 있게 다양한 시도를 해보세요!

조건 : 문장 3개 이상, 연결성이 있는 문장을 " " 으로 구분하여 ( )에 넣기

In [1]:
new_sentence = ("So cut the headlights, summer's a knife" "I'm always waiting for you just to cut to the bone" "Devils roll the dice, angels roll their eyes And if I bleed, you'll be the last to know")


In [4]:
new_word_set = list(set(new_sentence))
new_vocab = {char: i for i, char in enumerate(new_word_set, start=0)}

In [3]:
print(new_vocab)

{'f': 0, 'a': 1, ',': 2, 'A': 3, 'r': 4, 'm': 5, 'o': 6, 's': 7, 'k': 8, 'g': 9, 'v': 10, 't': 11, "'": 12, 'j': 13, 'l': 14, 'e': 15, 'w': 16, 'D': 17, 'u': 18, 'c': 19, 'h': 20, 'i': 21, ' ': 22, 'b': 23, 'd': 24, 'y': 25, 'I': 26, 'S': 27, 'n': 28}


In [5]:
print(new_word_set)

['f', 'a', ',', 'A', 'r', 'm', 'o', 's', 'k', 'g', 'v', 't', "'", 'j', 'l', 'e', 'w', 'D', 'u', 'c', 'h', 'i', ' ', 'b', 'd', 'y', 'I', 'S', 'n']


In [6]:
new_vocab_size = len(new_vocab)
print(new_vocab_size)

29


In [7]:
hidden_size = new_vocab_size
sequence_length = 10
learning_rate = 0.01

In [9]:
x_data = []
y_data = []

for i in range(0, len(new_sentence) - sequence_length):
  x_str = new_sentence[i:i+sequence_length]
  y_str = new_sentence[i+1:i+sequence_length+1]
  print(i, x_str, "->", y_str)

  # x_str과 y_str이 문자집합에 해당하는 인덱스를 각각 x_data, y_data에 append
  x_data.append([new_vocab[c] for c in x_str])
  y_data.append([new_vocab[d] for d in y_str])

0 So cut the -> o cut the 
1 o cut the  ->  cut the h
2  cut the h -> cut the he
3 cut the he -> ut the hea
4 ut the hea -> t the head
5 t the head ->  the headl
6  the headl -> the headli
7 the headli -> he headlig
8 he headlig -> e headligh
9 e headligh ->  headlight
10  headlight -> headlights
11 headlights -> eadlights,
12 eadlights, -> adlights, 
13 adlights,  -> dlights, s
14 dlights, s -> lights, su
15 lights, su -> ights, sum
16 ights, sum -> ghts, summ
17 ghts, summ -> hts, summe
18 hts, summe -> ts, summer
19 ts, summer -> s, summer'
20 s, summer' -> , summer's
21 , summer's ->  summer's 
22  summer's  -> summer's a
23 summer's a -> ummer's a 
24 ummer's a  -> mmer's a k
25 mmer's a k -> mer's a kn
26 mer's a kn -> er's a kni
27 er's a kni -> r's a knif
28 r's a knif -> 's a knife
29 's a knife -> s a knifeI
30 s a knifeI ->  a knifeI'
31  a knifeI' -> a knifeI'm
32 a knifeI'm ->  knifeI'm 
33  knifeI'm  -> knifeI'm a
34 knifeI'm a -> nifeI'm al
35 nifeI'm al -> ifeI'm alw
36

In [12]:
x_one_hot = [np.eye(new_vocab_size)[x_seq] for x_seq in x_data]
x_one_hot1 = [np.identity(new_vocab_size)[x_seq] for x_seq in x_data]

X = torch.FloatTensor(x_one_hot)
Y = torch.LongTensor(y_data)

<ipython-input-12-195c1967cb9b>:4: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:261.)
  X = torch.FloatTensor(x_one_hot)


In [13]:
class Net(torch.nn.Module):
  def __init__(self, input_dim, hidden_dim, layers):
    super(Net, self).__init__()
    self.rnn = torch.nn.RNN(input_dim, hidden_dim, num_layers=layers, batch_first=True)
    self.fc = torch.nn.Linear(hidden_dim, hidden_dim, bias=True)

  def forward(self, x):
    x, _status = self.rnn(x)
    x = self.fc(x)
    return x

In [14]:
net = Net(new_vocab_size, hidden_size, 2)

In [15]:
##12. loss function
criterion = torch.nn.CrossEntropyLoss()

##13. optimizer
optimizer = optim.Adam(net.parameters(), learning_rate)

In [19]:
##15. Training 시작

for i in range(100): # 100번의 epoch
    optimizer.zero_grad() # 매 반복마다 그래디언트 초기화
    outputs = net(X) # 모델의 출력 계산
    ##문제(7) : outputs, Y 형태 그대로 넣으면 안되죠. view 함수를 이용해 loss값을 계산해봅시다.
    loss = criterion(outputs.view(-1, new_vocab_size), Y.view(-1)) # view 함수로 텐서 형태 변환, view(-1)로 펼쳐주기
    loss.backward() # 역전파
    optimizer.step() # 최적화 알고리즘 사용, 파라미터 업데이트

    #16. 예측결과 확인
    results = outputs.argmax(dim=2)
    predict_str = ""
    for j, result in enumerate(results):
        if j == 0: # 처음에는 예측 결과를 전부 가져오기
            predict_str += ''.join([new_world_set[t] for t in result])
        else: # 그 다음에는 마지막 글자만 반복 추가
            predict_str += new_world_set[result[-1]]

    print(predict_str)

e     e e  ee  e ee e e  eee ee    e ee eee  e  e       e  e   e  e   e e e  e e  e   eeeee   e  eee  e   ee     ee e  eee  e e  eee  eee  e   eeee   ee  ee  e  e   e  e e  ee
        e                                                                         e                   e                     e                                    e             
                                                                                                                                                                               
                                                                                                                                                                               
                                                                                                                                                                               
                                                                                                                        

In [20]:
predict_str

"o cut the headlights, summer's a knifeI'm always waiting for you just to cut to the boneDevils roll the dice, angels roll the r eyes And if I bleed, you'll be the last to know"